In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)

2024-12-06 17:05:15.163259: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-06 17:05:15.166788: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-06 17:05:15.175668: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-06 17:05:15.188693: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-06 17:05:15.193121: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-06 17:05:15.204922: I tensorflow/core/platform/cpu_feature_gu

In [3]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

I0000 00:00:1733493101.470860   32843 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-12-06 16:51:41.588699: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2343] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [2]:
inputs = tf.keras.Input(shape=(200, 200, 3))

conv = tf.keras.layers.Conv2D(
    filters=32,
    kernel_size=(3, 3),
    activation='relu'
)(inputs)

pool = tf.keras.layers.MaxPooling2D(
    pool_size=(2, 2)
)(conv)

flat = tf.keras.layers.Flatten()(pool)

hidden = tf.keras.layers.Dense(
    units=64,
    activation='relu'
)(flat)

outputs = tf.keras.layers.Dense(
    units=1,
    activation='sigmoid'
)(hidden)

model = tf.keras.Model(inputs, outputs)

I0000 00:00:1733493917.662902   37720 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-12-06 17:05:17.695554: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2343] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [3]:
model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 200, 200, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 198, 198, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 99, 99, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 313632)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │    20,072,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 20,073,473 (76.57 MB)

 Trainable params: 20,073,473 (76.57 MB)

 Non-trainable params: 0 (0.00 B)

In [4]:
train_gen = ImageDataGenerator(rescale=1/255)

train_ds = train_gen.flow_from_directory(
    './data/train',
    class_mode='binary',
    target_size=(200, 200),
    batch_size=20,
    shuffle=True
)

test_gen = ImageDataGenerator(rescale=1/255)

test_ds = test_gen.flow_from_directory(
    './data/test',
    class_mode='binary',
    target_size=(200, 200),
    batch_size=20,
    shuffle=True
)

Found 800 images belonging to 2 classes.
Found 201 images belonging to 2 classes.


In [5]:
model.compile(
    optimizer=tf.keras.optimizers.SGD(learning_rate=0.002, momentum=0.8),
    loss='binary_crossentropy',
    metrics=['accuracy']
)


In [6]:
history = model.fit(
    train_ds,
    epochs=10,
    validation_data=test_ds
)

Epoch 1/10


/home/sam/Documents/projects/practice/courses/ml_zoomcamp/ml_zoom/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


40/40 ━━━━━━━━━━━━━━━━━━━━ 11s 266ms/step - accuracy: 0.5594 - loss: 0.7102 - val_accuracy: 0.6219 - val_loss: 0.6513
Epoch 2/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 10s 249ms/step - accuracy: 0.6731 - loss: 0.6022 - val_accuracy: 0.6169 - val_loss: 0.6166
Epoch 3/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 11s 274ms/step - accuracy: 0.6886 - loss: 0.5816 - val_accuracy: 0.6368 - val_loss: 0.6189
Epoch 4/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 11s 282ms/step - accuracy: 0.7103 - loss: 0.5564 - val_accuracy: 0.6517 - val_loss: 0.6105
Epoch 5/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 10s 251ms/step - accuracy: 0.7412 - loss: 0.5060 - val_accuracy: 0.6468 - val_loss: 0.6203
Epoch 6/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 11s 265ms/step - accuracy: 0.7516 - loss: 0.5079 - val_accuracy: 0.6667 - val_loss: 0.5996
Epoch 7/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 10s 259ms/step - accuracy: 0.7653 - loss: 0.4768 - val_accuracy: 0.6418 - val_loss: 0.6548
Epoch 8/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 10s 253ms/step - accuracy: 0.7833 - loss: 0.4518 - val_accuracy: 0.621

In [7]:
history_dict = history.history

# Calculate std dev for each metric
for metric in history_dict.keys():
    values = history_dict[metric]
    median = np.median(values)
    print(f'Median for {metric}: {median:.4f}')


Median for accuracy: 0.7325
Median for loss: 0.5216
Median for val_accuracy: 0.6443
Median for val_loss: 0.6177


In [8]:
# Convert history to a dictionary if it's not already

# Calculate std dev for each metric
for metric in history_dict.keys():
    values = history_dict[metric]
    std_dev = np.std(values)
    print(f'Standard deviation for {metric}: {std_dev:.4f}')

Standard deviation for accuracy: 0.0556
Standard deviation for loss: 0.0687
Standard deviation for val_accuracy: 0.0158
Standard deviation for val_loss: 0.0231


In [14]:
train_gen = ImageDataGenerator(
    rotation_range=50,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest',
    rescale=1/255
)

train_ds = train_gen.flow_from_directory(
    './data/train',
    target_size=(200, 200),
    batch_size=20,
    class_mode='binary',  # Add this line
    shuffle=True
)

val_gen = ImageDataGenerator(rescale=1/255)

val_ds = val_gen.flow_from_directory(
    './data/test',
    target_size=(200, 200),
    batch_size=20,
    class_mode='binary',  # Add this line
    shuffle=True
)

Found 800 images belonging to 2 classes.
Found 201 images belonging to 2 classes.


In [15]:

history = model.fit(
    train_ds,
    epochs=10,
    validation_data=val_ds
)

Epoch 1/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 14s 342ms/step - accuracy: 0.6299 - loss: 0.6467 - val_accuracy: 0.7114 - val_loss: 0.5777
Epoch 2/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 14s 340ms/step - accuracy: 0.6759 - loss: 0.6010 - val_accuracy: 0.7164 - val_loss: 0.5922
Epoch 3/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 14s 339ms/step - accuracy: 0.6822 - loss: 0.5763 - val_accuracy: 0.7214 - val_loss: 0.5791
Epoch 4/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 13s 330ms/step - accuracy: 0.6939 - loss: 0.5816 - val_accuracy: 0.6965 - val_loss: 0.5533
Epoch 5/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 13s 330ms/step - accuracy: 0.6949 - loss: 0.5748 - val_accuracy: 0.7114 - val_loss: 0.5765
Epoch 6/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 13s 327ms/step - accuracy: 0.7208 - loss: 0.5692 - val_accuracy: 0.7114 - val_loss: 0.5541
Epoch 7/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 13s 326ms/step - accuracy: 0.7010 - loss: 0.5703 - val_accuracy: 0.7164 - val_loss: 0.5405
Epoch 8/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 13s 329ms/step - accuracy: 0.7227 - loss: 0.5599 - val_accu

In [16]:
# Convert history to a dictionary if it's not already
history_dict = history.history
for metric in history_dict.keys():
    values = history_dict[metric] # Get only the last 5 values
    mean = np.mean(values)
    print(f'Mean for {metric}: {mean:.4f}')


Mean for accuracy: 0.6972
Mean for loss: 0.5824
Mean for val_accuracy: 0.7194
Mean for val_loss: 0.5616


In [17]:


# Calculate std dev for last 5 epochs of each metric
for metric in history_dict.keys():
    values = history_dict[metric][-5:]  # Get only the last 5 values
    mean = np.mean(values)
    print(f'Mean for {metric} (last 5 epochs): {mean:.4f}')


Mean for accuracy (last 5 epochs): 0.7177
Mean for loss (last 5 epochs): 0.5677
Mean for val_accuracy (last 5 epochs): 0.7274
Mean for val_loss (last 5 epochs): 0.5474
